In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install openpyxl
!pip3 install requests
!pip3 install beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage content
url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Find the table containing the list of international airports
tables = soup.find_all("table", {"class": "wikitable"})

# Extract the data from the table
data = []
for table in tables:
    rows = table.find_all("tr")
    i = 1
    while i < len(rows):
        row = rows[i]
        cols = row.find_all("td")
        if cols[0].get("rowspan"):
            rowspan = int(cols[0].get("rowspan"))
            location = cols[0].text.strip()
            airport = cols[1].text.strip()
            iata_code = cols[2].text.strip()
            data.append([location, airport, iata_code])

            # Extract the remaining rows for the same location
            for j in range(1, rowspan):
                sub_row = rows[i + j]
                sub_cols = sub_row.find_all("td")
                airport = sub_cols[0].text.strip()
                iata_code = sub_cols[1].text.strip()
                data.append([location, airport, iata_code])
            i += rowspan
        else:
            location = cols[0].text.strip()
            airport = cols[1].text.strip()
            iata_code = cols[2].text.strip()
            data.append([location, airport, iata_code])
            i += 1

# Create a DataFrame
df = pd.DataFrame(data, columns=["location", "airport", "iata_code"])

In [4]:
df

,location,airport,iata_code
0,Algiers,Houari Boumediene Airport,ALG
1,Annaba,Rabah Bitat Airport,AAE
2,Batna,Batna Airport,BLJ
3,Béjaïa,Soummam Airport,BJA
4,Chlef,Chlef International Airport,CFK
...,...,...,...
1485,Funafuti,Funafuti International Airport,FUN
1486,Luganville,Santo-Pekoa International Airport,SON
1487,Port Vila,Bauerfield International Airport,VLI
1488,Futuna,Pointe Vele Airport,FUT


In [ ]:
# Save to CSV
df.to_csv("international_airports.csv", index=False)